In [ ]:
def clustering_data(data, clustering_method = 'AffinityPropagation', plot  = True, dim_fold  = 'PCA', *kwargs) : 
    
    methods = [AffinityPropagation(), Birch(), HDBSCAN(min_cluster_size=3, max_cluster_size = 10, store_centers= 'both')]
    methods_str = ['AffinityPropagation', 'Birch', 'HDBSCAN']
    
    
    dim_fold_method = [PCA(2), Isomap(n_neighbors= 4, n_components=2)]
    dim_fold_method_str = ['PCA', 'Isomap']
    
    if dim_fold not in dim_fold_method_str : raise Exception('Unknown folding methods, availaible one : ' + str(dim_fold_method_str ))
    
    
    
    if clustering_method not in methods_str : raise Exception('Unknown clustering methods, availaible one : ' + str(methods_str))
    

    method = methods[methods_str.index(clustering_method)]
    print(method)
    method.fit(data)
    print(method.labels_.max())
    
    
    dim_fold = dim_fold_method[dim_fold_method_str.index(dim_fold)]
    data = dim_fold.fit_transform(data)
    
    elements_link = unique_indexes(matched_nodes[:,2])
    
    if plot : 

        fig, ax = plt.subplots()
        X, Y = data[:, 0], data[:, 1]
        mask = matched_nodes[:,2] == '0'
        scat = ax.scatter(X[mask],Y[mask], s = 5, c = method.labels_[mask].astype(np.uint32), cmap = cmap1)

        print( method.labels_[mask])
        ax.plot(X[mask], Y[mask], color = 'k', lw = .4)
            
        plt.colorbar(scat)
    
    return method, dim_fold

In [ ]:
def cluster_acc_(y_pred, y_true):
    """
    Calculate clustering accuracy.

    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
        
    """

    assert len(y_pred) == len(y_true)
    
    def neighboors(arr) : 
        
        ng = [np.arange(len(arr))[arr == arr[k]] for k in range(len(arr))]
        
        return ng
    
    ng_true = neighboors(y_true)
    ng_pred = neighboors(y_pred)
    metrics = np.mean([len(list(set(ng_pred[k]) & set(ng_true[k])))/ max(len(ng_true[k]), len(ng_pred[k])) for k in range(len(y_pred))])

    return metrics

In [ ]:
def cluster_acc(method, X, y_true):
    """
    Calculate clustering accuracy.

    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
        
    """
    method.fit(X)
    y_pred = method.labels_
    assert len(y_pred) == len(y_true)
    
    def neighboors(arr) : 
        
        ng = [np.arange(len(arr))[arr == arr[k]] for k in range(len(arr))]
        
        return ng
    ng_true = neighboors(y_true)
    ng_pred = neighboors(y_pred)
    metrics = np.mean([len(list(set(ng_pred[k]) & set(ng_true[k])))/ max(len(ng_true[k]), len(ng_pred[k])) for k in range(len(y_pred))])

    return metrics


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
method = HDBSCAN(alpha = .1, min_cluster_size=2,max_cluster_size=20, n_jobs = - 1)
method.fit(data2.reshape((len(data2),3 * len(data2) )))

In [ ]:
cluster_acc_(method.labels_, y_true)

In [ ]:
method = HDBSCAN(min_cluster_size=2,n_jobs = -1)

In [ ]:
param_grid = [{'cluster_selection_epsilon': np.linspace(0,1, 5, endpoint= False), 'max_cluster_size' : [17, 19, 22], 'alpha' : np.linspace(0.1,2,8), }]
clf = GridSearchCV(method, param_grid, scoring=cluster_acc, verbose = 3)
clf.fit(data2.reshape((len(data2),3 * len(data2) )), y_true)